# PCA
In this notebook, we conduct a PCA on our concat features. We want to check if the explained variance of the principal components fit our hypothesis that the image embeddings do not add any significant information.

We will conduct a PCA for
- the training split
- all splits combined

## 0. Imports and Constants

In [1]:
# AUTORELOAD
%load_ext autoreload
%autoreload 2

# GENERAL IMPORTS
import numpy as np
import pandas as pd

# TASK-SPECIFIC IMPORTS
from src import utils
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# CONSTANTS
users = ["patriziopalmisano", "onurdenizguler", "jockl"]
TRAIN = "train"
DEV = "dev"
TEST = "test"

####################### SELECT ###########################
user = users[2] # SELECT USER
version = "v2" # SELECT DATASET VERSION
dataset_version = version
##########################################################

if user in users[:2]:
    data_dir = f"/Users/{user}/Library/CloudStorage/GoogleDrive-check.worthiness@gmail.com/My Drive/data/CT23_1A_checkworthy_multimodal_english_{version}"
    cw_dir = f"/Users/{user}/Library/CloudStorage/GoogleDrive-check.worthiness@gmail.com/My Drive/"

else:
    data_dir = f"/home/jockl/Insync/check.worthiness@gmail.com/Google Drive/data/CT23_1A_checkworthy_multimodal_english_{dataset_version}"
    cw_dir = "/home/jockl/Insync/check.worthiness@gmail.com/Google Drive"

features_dir = f"{data_dir}/features"
labels_dir = f"{data_dir}/labels"
models_dir = f"{cw_dir}/models/vanillann"

## 1. Train Split

### 1.1 Load Features

Let's first load all the features and compare their shapes and contents with the original embeddings. We want to make sure that the first 768 feature dimensions indeed belong to the text embeddings and the last 768 to the image embeddings.

In [2]:
train_txt_emb, train_img_emb = utils.get_embeddings_from_pickle_file(f"{data_dir}/embeddings_{TRAIN}_{dataset_version}.pickle")
train_concat_features = np.load(f"{features_dir}/concat/concat_{TRAIN}_{dataset_version}.pickle", allow_pickle=True)
print(f"Train txt embeddings: {train_txt_emb.shape}")
print(f"Train img embeddings: {train_img_emb.shape}")
print(f"Train concat features: {train_concat_features.shape}")

Train txt embeddings: (2356, 768)
Train img embeddings: (2356, 768)
Train concat features: (2356, 1536)


Spot check if the first 768 feature dimensions indeed belong to the text embeddings, the latter 768 to the image embeddings:

In [3]:
print(f"Train txt embd excerpt: {train_txt_emb[0][:5]}")
print(f"Train features excerpt: {train_concat_features[0][:5]}")
print(f"Train img embd excerpt: {train_img_emb[0][-5:]}")
print(f"Train features excerpt: {train_concat_features[0][-5:]}")

Train txt embd excerpt: [ 0.31258187  0.8622302  -0.19572662  0.41690043 -0.8305622 ]
Train features excerpt: [ 0.31258187  0.8622302  -0.19572662  0.41690043 -0.8305622 ]
Train img embd excerpt: [ 0.20370999  0.39563796 -0.41939157 -0.35091972  0.02099419]
Train features excerpt: [ 0.20370999  0.39563796 -0.41939157 -0.35091972  0.02099419]


## 1.2 Normalize the Features

To perform a PCA, we first need to normalize the feature values. The normalized features should have a mean of 0, and standard deviation of 1.

In [4]:
train_normalized_concat_features = StandardScaler().fit_transform(train_concat_features)
print(f"Train normalized concat features: {train_normalized_concat_features.shape}")
print(f"Mean: {np.mean(train_normalized_concat_features)}")
print(f"Standard Deviation: {np.std(train_normalized_concat_features)}")

Train normalized concat features: (2356, 1536)
Mean: -3.0358901503824143e-10
Standard Deviation: 1.0


Mean and standard deviation have the desired values, the features are now normalized.

## 1.3 PCA and Explained Variance

Now that we have normalized feature values, we can compute all principal components.

IMPORTANT NOTE: There is no direct "mapping" between the n-th PC and the n-th feature dimension. The PCs are strictly ordered according to their explained variance values - by definition, the first PC explains the highest amount of variance, while this of course does not have to be the case for the first feature.

In [5]:
train_pca = PCA()
train_principal_components = train_pca.fit_transform(train_normalized_concat_features)
train_principal_components_df = pd.DataFrame(train_principal_components)
train_principal_components_df.tail()

,0,1,2,3,4,5,6,7,8,9,...,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535
2351,-6.303586,-10.861704,8.986289,0.001850,1.282223,4.173169,1.159974,-8.352036,2.054846,-0.449837,...,-0.000048,0.000346,0.000732,0.000309,-0.000086,0.000094,0.000246,0.000036,-0.000005,1.306930e-06
2352,-5.564433,15.178924,15.095158,-1.164485,1.757364,7.351459,4.915713,2.866526,-2.022675,-0.501234,...,-0.000185,-0.000296,-0.000203,0.000185,-0.000216,0.000020,0.000196,0.000328,-0.000095,-5.680644e-07
2353,-8.414557,-7.164299,-3.135854,1.749057,2.090608,0.284795,-1.228448,0.205026,-0.805630,-0.400631,...,-0.000015,-0.001139,0.000267,0.000211,0.000527,0.000028,-0.000058,-0.000009,0.000033,4.301229e-07
2354,-6.685371,13.612117,5.393781,-2.223035,14.665219,10.999657,-3.644086,6.255527,-3.607357,-2.052888,...,-0.000020,0.001407,-0.000715,-0.000512,-0.000194,0.001287,0.000310,0.000221,0.000010,2.617451e-06
2355,-4.898365,12.677189,4.458490,-3.420100,4.512700,5.863922,-2.172963,4.598461,-3.228651,5.265623,...,-0.000161,0.000900,0.000517,-0.000776,-0.000191,0.000158,0.000248,-0.000141,-0.000029,3.767290e-06


Sanity Check: Does the explained variance array have the right shape, do the values add up to 1?

In [15]:
train_explained_variance = train_pca.explained_variance_ratio_
sum_of_train_explained_variance_values = np.sum(train_explained_variance)
print(f"Explained variance array shape: {train_explained_variance.shape}")
print(f"Sum of all explained variance values: {sum_of_train_explained_variance_values}")

Explained variance array shape: (1536,)
Sum of all explained variance values: 1.0000001192092896


Now, we have the explained variance for all the 1536 principal components. Let's now sum over the first and last 768 values:

In [7]:
train_txt_features_explained_variance = np.sum(train_explained_variance[:train_txt_emb.shape[1]])
train_img_features_explained_variance = np.sum(train_explained_variance[-train_img_emb.shape[1]:])
print(f"Explained variance of the first 768 PCs within train split: {train_txt_features_explained_variance}")
print(f"Explained variance of the last 768 PCs within train split: {train_img_features_explained_variance}")

Explained variance of the first 768 PCs within train split: 0.9474944472312927
Explained variance of the last 768 PCs within train split: 0.052505627274513245


# 2. All Splits

### 2.1 Load Features
Let's first load all the features and compare their shapes and contents with the original embeddings. We want to make sure that the first 768 feature dimensions indeed belong to the text embeddings and the last 768 to the image embeddings.

In [8]:
dev_concat_features = np.load(f"{features_dir}/concat/concat_{DEV}_{dataset_version}.pickle", allow_pickle=True)
test_concat_features = np.load(f"{features_dir}/concat/concat_{TEST}_{dataset_version}.pickle", allow_pickle=True)
all_concat_features = np.concatenate((train_concat_features, dev_concat_features, test_concat_features))
print(f"All concat features: {all_concat_features.shape}")


All concat features: (3175, 1536)


Spot check if the first 768 feature dimensions indeed belong to the text embeddings, the latter 768 to the image embeddings:

In [9]:
# Load test embeddings
test_txt_emb, test_img_emb = utils.get_embeddings_from_pickle_file(f"{data_dir}/embeddings_{TEST}_{dataset_version}.pickle")

# Spot check
print(f"Test txt embd excerpt: {test_txt_emb[-1][:5]}")
print(f"Test features excerpt: {all_concat_features[-1][:5]}")
print(f"Test img embd excerpt: {test_img_emb[-1][-5:]}")
print(f"Test features excerpt: {all_concat_features[-1][-5:]}")

Test txt embd excerpt: [ 0.6375199  -0.53175956  0.20533158 -0.59946156  0.6934856 ]
Test features excerpt: [ 0.6375199  -0.53175956  0.20533158 -0.59946156  0.6934856 ]
Test img embd excerpt: [-0.6128674   0.13787843 -0.59581023  0.30937177  0.3236546 ]
Test features excerpt: [-0.6128674   0.13787843 -0.59581023  0.30937177  0.3236546 ]


## 2.2 Normalize the Features

To perform a PCA, we first need to normalize the feature values. The normalized features should have a mean of 0, and standard deviation of 1.

In [10]:
all_normalized_concat_features = StandardScaler().fit_transform(all_concat_features)
print(f"Dev normalized concat features: {all_normalized_concat_features.shape}")
print(f"Mean: {np.mean(all_normalized_concat_features)}")
print(f"Standard Deviation: {np.std(all_normalized_concat_features)}")

Dev normalized concat features: (3175, 1536)
Mean: 1.6739362673767744e-10
Standard Deviation: 1.0


Mean and standard deviation have the desired values, the features are now normalized.

## 2.3 PCA and Explained Variance

Now that we have normalized feature values, we can compute all principal components.

IMPORTANT NOTE: There is no direct "mapping" between the n-th PC and the n-th feature dimension. The PCs are strictly ordered according to their explained variance values - by definition, the first PC explains the highest amount of variance, while this of course does not have to be the case for the first feature.

In [11]:
all_pca = PCA()
all_principal_components = all_pca.fit_transform(all_normalized_concat_features)
all_principal_components_df = pd.DataFrame(all_principal_components)
all_principal_components_df.tail()

,0,1,2,3,4,5,6,7,8,9,...,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535
3170,-0.509420,-6.539375,3.930840,-6.451927,-5.348428,-2.384798,-0.779864,-5.852808,2.231250,-3.941541,...,0.001402,-0.000920,-0.000160,0.000540,-0.000198,0.000056,0.000084,0.000050,-0.000023,-1.196255e-06
3171,-4.458532,-14.868099,10.573829,-10.466908,0.568076,6.589172,2.355393,7.669797,-1.359096,-4.657529,...,0.000227,-0.000316,0.000413,0.000144,0.000721,-0.000532,0.000470,-0.000016,-0.000009,-3.934614e-06
3172,-4.387179,-4.060971,-7.321678,2.225588,2.264590,1.069938,0.334720,-3.195267,-2.207384,-5.335334,...,0.000872,-0.000283,-0.000864,-0.000395,-0.000498,-0.000170,0.000033,-0.000263,-0.000018,-3.317778e-06
3173,-4.627469,-6.619272,-8.359013,4.613657,0.688303,-0.457603,-1.968335,-3.471986,0.361107,-5.644895,...,-0.000321,-0.000440,0.001092,-0.000409,-0.000395,-0.000395,0.000123,0.000199,0.000023,-1.768249e-06
3174,-5.531907,-3.422538,-4.801727,-0.755215,-3.030000,-4.084527,0.745827,2.085324,-1.728323,-0.899399,...,0.000189,-0.000214,-0.000448,0.000365,0.000415,0.000190,-0.000121,-0.000129,-0.000018,-5.678798e-07


Sanity Check: Has the explained variance array the right shape, do the values add up to 1?

In [16]:
all_explained_variance = all_pca.explained_variance_ratio_
all_sum_of_explained_variance_values = np.sum(all_explained_variance)
print(f"Explained variance array shape: {all_explained_variance.shape}")
print(f"Sum of all explained variance values: {all_sum_of_explained_variance_values}")

Explained variance array shape: (1536,)
Sum of all explained variance values: 1.0000001192092896


Now, we have the explained variance for all the 1536 principal components. Let’s now sum over the first and last 768 principal components:

In [13]:
all_txt_features_explained_variance = np.sum(all_explained_variance[:train_txt_emb.shape[1]])
all_img_features_explained_variance = np.sum(all_explained_variance[-train_img_emb.shape[1]:])
print(f"Explained variance of the first 768 PCs within all splits: {all_txt_features_explained_variance}")
print(f"Explained variance of the last 768 PCs within all splits: {all_img_features_explained_variance}")

Explained variance of the first 768 PCs within all splits: 0.9348670840263367
Explained variance of the last 768 PCs within all splits: 0.0651329830288887


# 3. Summary of Results and Conclusion

Results for train split and all data:

In [14]:
print(f"Explained variance of text embeddings within train split: {train_txt_features_explained_variance}")
print(f"Explained variance of img embeddings within train split: {train_img_features_explained_variance}\n")

print(f"Explained variance of text embeddings within all splits: {all_txt_features_explained_variance}")
print(f"Explained variance of img embeddings within all splits: {all_img_features_explained_variance}")

Explained variance of text embeddings within train split: 0.9474944472312927
Explained variance of img embeddings within train split: 0.052505627274513245

Explained variance of text embeddings within all splits: 0.9348670840263367
Explained variance of img embeddings within all splits: 0.0651329830288887


- The first 768 PCs capture around 95 % of the features' variance.
- Even though the first PCs do not mathematically correspond to the first 768 features (i.e. the text embeddings), this fits our hypothesis.
- Almost all the variance can be explained by half the number of dimensions - and half our dimensions are made up by image embedding dimensions.
- This matches our previous findings: Training an SVM/VanillaNN on text only yields hardly worse results than training on the concat features.